## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [7]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/TOBS_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [8]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.207785,-0.186461,-0.051425,-0.111689,-0.119900,-0.012736,-0.066613,0.004925,0.082807,-0.000261
1,0.044875,0.017570,0.025454,0.039794,0.009284,-0.030974,0.043010,0.081765,0.054003,-0.009176
2,0.044875,0.017570,0.025454,0.039794,0.009284,-0.030974,0.043010,0.081765,0.054003,-0.009176
3,-0.075543,-0.379706,-0.126418,-0.041366,-0.178670,-0.151065,0.029473,-0.045299,-0.172898,-0.008071
4,0.047835,0.009280,0.049322,0.066063,0.071917,-0.080153,0.114351,-0.248144,0.322055,0.288027


In [9]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00093578', u'USC00090586', u'USC00084892', u'USC00093516', u'USC00090140']


,0,1,2,3,4,5,6,7,8,9
station,,,,,,,,,,
USC00093578,-0.207785,-0.186461,-0.051425,-0.111689,-0.119900,-0.012736,-0.066613,0.004925,0.082807,-0.000261
USC00090586,0.044875,0.017570,0.025454,0.039794,0.009284,-0.030974,0.043010,0.081765,0.054003,-0.009176
USC00084892,0.044875,0.017570,0.025454,0.039794,0.009284,-0.030974,0.043010,0.081765,0.054003,-0.009176
USC00093516,-0.075543,-0.379706,-0.126418,-0.041366,-0.178670,-0.151065,0.029473,-0.045299,-0.172898,-0.008071
USC00090140,0.047835,0.009280,0.049322,0.066063,0.071917,-0.080153,0.114351,-0.248144,0.322055,0.288027


In [10]:
table=Eig.join(Stations,how='left')

In [11]:
table=table[['name','latitude','longitude','elevation',0,1,2,3]]#,4,5,6,7]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00093578,FT GAINES 2,31.6150,-85.0494,64.0,-0.207785,-0.186461,-0.051425,-0.111689
USC00090586,BAINBRIDGE INTL PAPER,30.8228,-84.6175,57.9,0.044875,0.017570,0.025454,0.039794
USC00084892,LAMONT 6 WNW,30.4167,-83.9167,64.0,0.044875,0.017570,0.025454,0.039794
USC00093516,FT GAINES,31.6000,-85.0500,103.9,-0.075543,-0.379706,-0.126418,-0.041366


In [12]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [14]:
min_lat,max_lat,min_long,max_long = box = (29, 32, -86, -83)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [10]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined